<a href="https://colab.research.google.com/github/walar2/coins/blob/main/Coin_thb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Training the image dataset to recongnize the coin values

First, upload the dataset to the collab and unzip.

In [ ]:
!unzip -q "/content/coin.v1i.yolov8.zip" -d "/content"

After unziping the datafile, make sure that the dependencies needed are installed properly. Since this model will be trained using Yolo, we will be installing ultralytics.

In [ ]:
# === YOLOv8 training on a T4 GPU in ONE Colab cell (all under /content) ===

# 1) Install
!pip -q install --upgrade "ultralytics>=8.2.0"

# 2) User knobs (edit as needed)
MODEL     = "yolov8s.pt"   # yolov8n/s/m/l/x .pt or a *.yaml to train from scratch
EPOCHS    = 100
IMG_SIZE  = 640
BATCH     = -1             # <= AutoBatch for T4 (finds the largest safe batch)
WORKERS   = None           # will set to min(4, max(2, os.cpu_count()//2)) below
AUGMENT   = True
PROJECT   = "runs/train"
RUN_NAME  = "yolov8_custom_t4"

# 3) Paths (fixed to /content)
import os, sys, pathlib, textwrap, yaml, glob, subprocess, json
from datetime import datetime
import torch
from ultralytics import YOLO

ROOT = "/content"
DATA_YAML = f"{ROOT}/data.yaml"

TRAIN_IMAGES = f"{ROOT}/train/images"
TRAIN_LABELS = f"{ROOT}/train/labels"

# Accept either /content/val or /content/valid (prefer val if both exist)
VAL_ROOT = f"{ROOT}/val" if os.path.isdir(f"{ROOT}/val") else (f"{ROOT}/valid" if os.path.isdir(f"{ROOT}/valid") else f"{ROOT}/val")
VAL_IMAGES = f"{VAL_ROOT}/images"
VAL_LABELS = f"{VAL_ROOT}/labels"

TEST_IMAGES = f"{ROOT}/test/images" if os.path.isdir(f"{ROOT}/test/images") else None
TEST_LABELS = f"{ROOT}/test/labels" if os.path.isdir(f"{ROOT}/test/labels") else None

print("Torch:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    try:
        # Show GPU details (T4 expected)
        _ = subprocess.run(["nvidia-smi", "--query-gpu=name,memory.total", "--format=csv,noheader"], check=True, capture_output=True, text=True)
        print("nvidia-smi:", _.stdout.strip())
    except Exception as e:
        print("nvidia-smi not available:", e)

    print("GPU:", torch.cuda.get_device_name(0))

# T4-friendly perf toggles
torch.backends.cudnn.benchmark = True
try:
    torch.set_float32_matmul_precision("high")  # PyTorch 2.x
except Exception:
    pass

DEVICE = "0" if torch.cuda.is_available() else "cpu"
print("Using device:", DEVICE)

# Resolve workers (modest for Colab, persistent to reduce dataloader overhead)
if WORKERS is None:
    cpu = os.cpu_count() or 4
    WORKERS = min(4, max(2, cpu // 2))
print("Dataloader workers:", WORKERS)

def _exists_dir(p):
    return os.path.isdir(p)

# Sanity checks for directory structure
missing = []
for p in [TRAIN_IMAGES, TRAIN_LABELS, VAL_IMAGES, VAL_LABELS]:
    if not _exists_dir(p):
        missing.append(p)

if missing:
    raise FileNotFoundError(
        textwrap.dedent(
            f"""
            Some required directories were not found:
              {os.linesep.join(missing)}
            Expected YOLO format under /content:
              /content/train/images, /content/train/labels
              /content/val/images   OR /content/valid/images
              /content/val/labels   OR /content/valid/labels
            Optional:
              /content/test/images, /content/test/labels
            """
        )
    )

def resolve_model_arg(model_str: str, use_pretrained_default=True):
    p = pathlib.Path(model_str)
    s = str(p)
    if s.endswith(".pt") or s.endswith(".yaml"):
        return s
    return s + (".pt" if use_pretrained_default else ".yaml")

MODEL_ARG = resolve_model_arg(MODEL)
print("\nModel arg:", MODEL_ARG)
print("Train images:", TRAIN_IMAGES)
print("Val images:", VAL_IMAGES)
print("Test images:", TEST_IMAGES if TEST_IMAGES else "(none)")
print("Data YAML:", DATA_YAML)

# 4) If /content/data.yaml is missing, auto-create one that points to /content/*
def infer_num_classes_from_labels(label_dirs):
    ids = set()
    for d in label_dirs:
        if not d or not os.path.isdir(d):
            continue
        for fp in glob.glob(os.path.join(d, "*.txt")):
            try:
                with open(fp, "r") as f:
                    for line in f:
                        line = line.strip()
                        if not line:
                            continue
                        parts = line.split()
                        cid = int(float(parts[0]))  # robust parse
                        if cid >= 0:
                            ids.add(cid)
            except Exception:
                pass
    if not ids:
        return None  # unknown
    return max(ids) + 1

def maybe_create_yaml():
    if os.path.exists(DATA_YAML):
        print("\nFound existing data.yaml; using it as-is.")
        return

    print("\nNo data.yaml found — creating /content/data.yaml for you...")

    # Try to infer number of classes from labels; fallback to 1
    ncls = infer_num_classes_from_labels([TRAIN_LABELS, VAL_LABELS, TEST_LABELS])
    if ncls is None:
        ncls = 1
        print("Warning: Could not infer class count from labels; defaulting to 1 class (0: class_0).")
    else:
        print(f"Inferred number of classes: {ncls}")

    names = {i: f"class_{i}" for i in range(ncls)}

    data = {
        "path": ROOT,
        "train": "train/images",
        "val":   f"{pathlib.Path(VAL_ROOT).name}/images",  # 'val/images' or 'valid/images'
        "names": names
    }
    if TEST_IMAGES and os.path.isdir(TEST_IMAGES):
        data["test"] = "test/images"

    with open(DATA_YAML, "w") as f:
        yaml.safe_dump(data, f, sort_keys=False)
    print("Wrote:", DATA_YAML)
    print("\nIf you have real class names, edit /content/data.yaml and replace:")
    print(textwrap.indent("\n".join([f"{k}: {v}" for k, v in names.items()]), prefix="  "))

maybe_create_yaml()

# 5) Train
RUN_NAME_STAMPED = f"{RUN_NAME}_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
print(f"\n==> Initializing YOLO with: {MODEL_ARG}")
model = YOLO(MODEL_ARG)

print("\n==> Starting training on T4-optimized settings...")
results = model.train(
    data=DATA_YAML,
    epochs=EPOCHS,
    imgsz=IMG_SIZE,
    batch=BATCH,            # AutoBatch: largest that fits on T4
    device=DEVICE,          # "0" when CUDA; "cpu" otherwise
    workers=WORKERS,
    project=PROJECT,
    name=RUN_NAME_STAMPED,
    patience=20,
    verbose=True,
    save=True,
    plots=True,
    cache=True,             # cache images to RAM for speed
    augment=AUGMENT,
    amp=True,               # mixed precision for T4 Tensor Cores
    cos_lr=True,            # smooth cosine LR schedule (often helps)
    deterministic=False,    # allow fastest kernels
)

# 6) Validate best
print("\n==> Validating best checkpoint...")
metrics = model.val(data=DATA_YAML, imgsz=IMG_SIZE, batch=max(1, (BATCH if isinstance(BATCH, int) and BATCH > 0 else 16)//2), device=DEVICE, amp=True)
print("\nValidation metrics:", metrics)

# 7) Paths to artifacts
run_dir = os.path.join(PROJECT, RUN_NAME_STAMPED)
weights_best = os.path.join(run_dir, "weights", "best.pt")
weights_last = os.path.join(run_dir, "weights", "last.pt")

print("\n==> Training complete.")
print("Run directory:", run_dir)
print("Best weights: ", weights_best, "(exists:", os.path.exists(weights_best), ")")
print("Last weights: ", weights_last, "(exists:", os.path.exists(weights_last), ")")

# 8) (Optional) Export ONNX
# model = YOLO(weights_best if os.path.exists(weights_best) else MODEL_ARG)
# model.export(format="onnx")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 74.9 MB/s eta 0:00:00
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Torch: 2.8.0+cu126
CUDA available: True
nvidia-smi: Tesla T4, 15360 MiB
GPU: Tesla T4
Using device: 0
Dataloader workers: 4

Model arg: yolov8s.pt
Train images: /content/train/images
Val images: /content/valid/images
Test images: /content/test/images
Data YAML: /content/data.yaml

Found existing data.yaml; using it as-is.

==> Initializing YOLO with: yolov8s.pt

==> Starting training on T4-optimized settings...
Ultralytics 8.3.220 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=True, auto_augment=randaugment, batch=-1, bgr=0.0, box=7.5, 